In [1]:
from pyspark.sql import SparkSession
import getpass

usename = getpass.getuser()

spark = SparkSession.builder \
    .config("spark.ui.port", "4050") \
    .config("spark.sql.warehouse.dir" , "C:\data\Thuc_hanh") \
    .getOrCreate()

In [2]:
orders_rdd = spark.sparkContext.textFile("C:/data/Sample_Superstore.csv")

In [3]:
orders_rdd.count()

9995

In [4]:
orders_rdd.take(10)

['Row ID;Order ID;Order Date;Ship Date;Ship Mode;Customer ID;Customer Name;Segment;Country;City;State;Postal Code;Region;Product ID;Category;Sub-Category;Product Name;Sales;Quantity;Discount;Profit',
 '1;CA-2016-152156;08/11/2016;11/11/2016;Second Class;CG-12520;Claire Gute;Consumer;United States;Henderson;Kentucky;42420;South;FUR-BO-10001798;Furniture;Bookcases;Bush Somerset Collection Bookcase;261,96;2;0;41,9136',
 '2;CA-2016-152156;08/11/2016;11/11/2016;Second Class;CG-12520;Claire Gute;Consumer;United States;Henderson;Kentucky;42420;South;FUR-CH-10000454;Furniture;Chairs;Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back;731,94;3;0;219,582',
 '3;CA-2016-138688;12/06/2016;16/06/2016;Second Class;DV-13045;Darrin Van Huff;Corporate;United States;Los Angeles;California;90036;West;OFF-LA-10000240;Office Supplies;Labels;Self-Adhesive Address Labels for Typewriters by Universal;14,62;2;0;6,8714',
 "4;US-2015-108966;11/10/2015;18/10/2015;Standard Class;SO-20335;Sean O'Donnell;Cons

In [5]:
Segment_rdd = orders_rdd.map(lambda x : (x.split(";")[7],1))
Segment_rdd.take(10)

[('Segment', 1),
 ('Consumer', 1),
 ('Consumer', 1),
 ('Corporate', 1),
 ('Consumer', 1),
 ('Consumer', 1),
 ('Consumer', 1),
 ('Consumer', 1),
 ('Consumer', 1),
 ('Consumer', 1)]

In [6]:
Segment_reduced_rdd = Segment_rdd.reduceByKey(lambda x,y : x+y)
Segment_reduced_rdd.collect()

[('Consumer', 5191),
 ('Corporate', 3020),
 ('Home Office', 1783),
 ('Segment', 1)]

In [7]:
Segment_sorted_rdd = Segment_reduced_rdd.sortBy(lambda x : x[1])
Segment_sorted_rdd.collect()

[('Segment', 1),
 ('Home Office', 1783),
 ('Corporate', 3020),
 ('Consumer', 5191)]

In [8]:
Customer = orders_rdd.map(lambda x : (x.split(";")[6],1))
Customer.take(5)

[('Customer Name', 1),
 ('Claire Gute', 1),
 ('Claire Gute', 1),
 ('Darrin Van Huff', 1),
 ("Sean O'Donnell", 1)]

In [9]:
Customer_reduced_rdd = Customer.reduceByKey(lambda x,y : x+y)
Customer_reduced_rdd.collect()

[('Customer Name', 1),
 ('Darrin Van Huff', 9),
 ("Sean O'Donnell", 15),
 ('Brosina Hoffman', 24),
 ('Andrew Allen', 12),
 ('Harold Pawlan', 10),
 ('Zuschuss Donatelli', 9),
 ('Tracy Blumstein', 20),
 ('Matt Abelman', 34),
 ('Gene Hale', 3),
 ('Erin Smith', 20),
 ('Odella Nelson', 15),
 ("Patrick O'Donnell", 13),
 ('Lena Hernandez', 15),
 ('Janet Molinari', 13),
 ('Ted Butterfield', 18),
 ('Brendan Sweed', 10),
 ('Karen Daniels', 16),
 ('Henry MacAllister', 7),
 ('Ken Brennan', 9),
 ('Duane Noonan', 11),
 ('Julie Creighton', 9),
 ('Christopher Schild', 14),
 ('Gary Mitchum', 12),
 ('Roger Barcio', 7),
 ('Katherine Ducich', 10),
 ('Elpida Rittenbach', 5),
 ('Pete Armstrong', 7),
 ('Cynthia Voltz', 19),
 ('Clay Ludtke', 28),
 ('Greg Guthrie', 24),
 ('Troy Staebel', 11),
 ('Lindsay Shagiari', 22),
 ('Lena Creighton', 23),
 ('Sally Hughsby', 29),
 ('Helen Andreada', 18),
 ('Maureen Gastineau', 10),
 ('Justin Ellison', 7),
 ('Tamara Willingham', 12),
 ('Neil Knudson', 11),
 ('Dave Brooks', 

In [10]:
Customer_sorted_rdd = Customer_reduced_rdd.sortBy(lambda x : x[1], False)
Customer_sorted_rdd.take(10)

[('William Brown', 37),
 ('Matt Abelman', 34),
 ('Paul Prost', 34),
 ('John Lee', 34),
 ('Seth Vernon', 32),
 ('Jonathan Doherty', 32),
 ('Chloris Kastensmidt', 32),
 ('Edward Hooks', 32),
 ('Emily Phan', 31),
 ('Zuschuss Carroll', 31)]

In [11]:
Sub_Category = orders_rdd.map(lambda x : (x.split(";")[15],1))
Sub_Category.take(10)

[('Sub-Category', 1),
 ('Bookcases', 1),
 ('Chairs', 1),
 ('Labels', 1),
 ('Tables', 1),
 ('Storage', 1),
 ('Furnishings', 1),
 ('Art', 1),
 ('Phones', 1),
 ('Binders', 1)]

In [12]:
Sub_Category_reduced_rdd = Sub_Category.reduceByKey(lambda x,y : x+y)
Sub_Category_reduced_rdd.take(10)

[('Sub-Category', 1),
 ('Art', 796),
 ('Binders', 1523),
 ('Accessories', 775),
 ('Envelopes', 254),
 ('Fasteners', 217),
 ('Machines', 115),
 ('Bookcases', 228),
 ('Chairs', 617),
 ('Labels', 364)]

In [13]:
Sub_Category_sorted_rdd = Sub_Category_reduced_rdd.sortBy(lambda x : x[1], False)
Sub_Category_sorted_rdd.take(10)

[('Binders', 1523),
 ('Paper', 1370),
 ('Furnishings', 957),
 ('Phones', 889),
 ('Storage', 846),
 ('Art', 796),
 ('Accessories', 775),
 ('Chairs', 617),
 ('Appliances', 466),
 ('Labels', 364)]

In [14]:
Customer_total_rdd = orders_rdd.map(lambda x : x.split(";")[6])
Customer_total_rdd.take(10)

['Customer Name',
 'Claire Gute',
 'Claire Gute',
 'Darrin Van Huff',
 "Sean O'Donnell",
 "Sean O'Donnell",
 'Brosina Hoffman',
 'Brosina Hoffman',
 'Brosina Hoffman',
 'Brosina Hoffman']

In [15]:
total_rdd = Customer_total_rdd.distinct()
total_rdd.count()


794

In [16]:
filter_rdd = orders_rdd.filter(lambda x : (x.split(";")[14]) == 'Technology')
filter_rdd.take(5)

['8;CA-2014-115812;09/06/2014;14/06/2014;Standard Class;BH-11710;Brosina Hoffman;Consumer;United States;Los Angeles;California;90032;West;TEC-PH-10002275;Technology;Phones;Mitel 5320 IP Phone VoIP phone;907,152;6;0,2;90,7152',
 '12;CA-2014-115812;09/06/2014;14/06/2014;Standard Class;BH-11710;Brosina Hoffman;Consumer;United States;Los Angeles;California;90032;West;TEC-PH-10002033;Technology;Phones;Konftel 250 Conference\xa0phone\xa0- Charcoal black;911,424;4;0,2;68,3568',
 '20;CA-2014-143336;27/08/2014;01/09/2014;Second Class;ZD-21925;Zuschuss Donatelli;Consumer;United States;San Francisco;California;94109;West;TEC-PH-10001949;Technology;Phones;Cisco SPA 501G IP Phone;213,48;3;0,2;16,011',
 '27;CA-2016-121755;16/01/2016;20/01/2016;Second Class;EH-13945;Eric Hoffmann;Consumer;United States;Los Angeles;California;90049;West;TEC-AC-10003027;Technology;Accessories;Imation\xa08GB Mini TravelDrive USB 2.0\xa0Flash Drive;90,57;3;0;11,7741',
 '36;CA-2016-117590;08/12/2016;10/12/2016;First Class

In [17]:
Customer_Technology = filter_rdd.map(lambda x :(x.split(";")[6],1))
Customer_Technology.take(5)

[('Brosina Hoffman', 1),
 ('Brosina Hoffman', 1),
 ('Zuschuss Donatelli', 1),
 ('Eric Hoffmann', 1),
 ('Gene Hale', 1)]

In [18]:
Customer_Technology_reduced = Customer_Technology.reduceByKey(lambda x,y : x+y)
Customer_Technology_reduced.take(5)

[('Brosina Hoffman', 7),
 ('Zuschuss Donatelli', 3),
 ('Gene Hale', 1),
 ('Odella Nelson', 2),
 ('Lena Hernandez', 7)]

In [19]:
Customer_Technology_sorted = Customer_Technology_reduced.sortBy(lambda x : x[1], False)
Customer_Technology_sorted.take(1)

[('Laura Armstrong', 9)]